In [23]:
import pickle
import joblib
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [24]:
data = pd.read_csv("E:\Virtual_room\project\AIrecruiter\data\processed\\processed_resume.csv")

In [25]:
data.head()

,Resume_str,Category,cleaned_resume,lemmatize_data,tokenize_data
0,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,HR,hr administrator marketing associate ...,hr administrator marketing associate hr admini...,"['hr', 'administrator', 'marketing', 'associat..."
1,"HR SPECIALIST, US HR OPERATIONS ...",HR,hr specialist us hr operations ...,hr specialist u hr operation summary versatile...,"['hr', 'specialist', 'u', 'hr', 'operation', '..."
2,HR DIRECTOR Summary Over 2...,HR,hr director summary over ...,hr director summary year experience recruiting...,"['hr', 'director', 'summary', 'year', 'experie..."
3,HR SPECIALIST Summary Dedica...,HR,hr specialist summary dedica...,hr specialist summary dedicated driven dynamic...,"['hr', 'specialist', 'summary', 'dedicated', '..."
4,HR MANAGER Skill Highlights ...,HR,hr manager skill highlights ...,hr manager skill highlight hr skill hr departm...,"['hr', 'manager', 'skill', 'highlight', 'hr', ..."


In [26]:
data.describe()

,Resume_str,Category,cleaned_resume,lemmatize_data,tokenize_data
count,3446,3446,3446,3445,3446
unique,2648,45,2648,2647,2648
top,"Technical Skills Web Technologies: Angular JS,...",SALES,technical skills web technologies angular js ...,technical skill web technology angular j html ...,"['technical', 'skill', 'web', 'technology', 'a..."
freq,18,156,18,18,18


In [27]:
data.isnull().sum()

Resume_str        0
Category          0
cleaned_resume    0
lemmatize_data    1
tokenize_data     0
dtype: int64

In [28]:
data.dropna(inplace=True)

In [29]:
data.isnull().sum()

Resume_str        0
Category          0
cleaned_resume    0
lemmatize_data    0
tokenize_data     0
dtype: int64

In [30]:
tfidf = TfidfVectorizer(stop_words='english', max_features=3000)

In [31]:
from sklearn.preprocessing import LabelEncoder
lblencoder = LabelEncoder()
data['Category_lbl'] = lblencoder.fit_transform(data['Category'])
# target = pd.get_dummies(data['Category'])

In [32]:
datao = data[['Category','Category_lbl']].drop_duplicates(keep='first').sort_values('Category_lbl')

In [33]:
target_dict = {key:value for key, value in zip(data['Category_lbl'], data['Category'])}

In [34]:
target_dict

{31: 'HR',
 19: 'DESIGNER',
 32: 'INFORMATION_TECHNOLOGY',
 42: 'TEACHER',
 1: 'ADVOCATE',
 12: 'BUSINESS_DEVELOPMENT',
 30: 'HEALTHCARE',
 27: 'FITNESS',
 2: 'AGRICULTURE',
 10: 'BPO',
 40: 'SALES',
 16: 'CONSULTANT',
 21: 'DIGITAL_MEDIA',
 6: 'AUTOMOBILE',
 13: 'CHEF',
 26: 'FINANCE',
 3: 'APPAREL',
 24: 'ENGINEERING',
 0: 'ACCOUNTANT',
 15: 'CONSTRUCTION',
 38: 'PUBLIC_RELATIONS',
 8: 'BANKING',
 4: 'ARTS',
 7: 'AVIATION',
 17: 'DATA SCIENCE',
 44: 'WEB DESIGNING',
 34: 'MECHANICAL ENGINEER',
 29: 'HEALTH AND FITNESS',
 14: 'CIVIL ENGINEER',
 33: 'JAVA DEVELOPER',
 11: 'BUSINESS ANALYST',
 41: 'SAP DEVELOPER',
 5: 'AUTOMATION TESTING',
 23: 'ELECTRICAL ENGINEERING',
 36: 'OPERATIONS MANAGER',
 39: 'PYTHON DEVELOPER',
 20: 'DEVOPS ENGINEER',
 35: 'NETWORK SECURITY ENGINEER',
 37: 'PMO',
 18: 'DATABASE',
 28: 'HADOOP',
 25: 'ETL DEVELOPER',
 22: 'DOTNET DEVELOPER',
 9: 'BLOCKCHAIN',
 43: 'TESTING'}

In [35]:
features = data['lemmatize_data'].values
target = data['Category_lbl'].values

In [36]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(features,target,test_size=0.2,random_state=42, shuffle=True, stratify=target)

In [37]:
X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)

In [38]:
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
model1 = SVC()
model2 = LogisticRegression()


In [39]:
from sklearn.multiclass import OneVsRestClassifier
model_svc = OneVsRestClassifier(model1)
model_log = OneVsRestClassifier(model2)

In [40]:
model_svc.fit(X_train, y_train)
model_log.fit(X_train, y_train)

OneVsRestClassifier(estimator=LogisticRegression())

In [41]:
t_pred = model_svc.predict(X_train)
t_pred2 = model_svc.predict(X_test)
pred = model_log.predict(X_train)
pred2 = model_log.predict(X_test)


print("training accuracy of onevsall model svc:",accuracy_score(y_train,t_pred))
print("testing accuracy of onevsall model svc:",accuracy_score(y_test,t_pred2))
print("training accuracy of onevsall model log:",accuracy_score(y_train,pred))
print("testing accuracy of onevsall model log:",accuracy_score(y_test,pred2))

training accuracy of onevsall model svc: 0.9916545718432511
testing accuracy of onevsall model svc: 0.7808417997097242
training accuracy of onevsall model log: 0.8276487663280117
testing accuracy of onevsall model log: 0.7576197387518142


In [42]:
print("classifaction report for svc:", classification_report(y_test, t_pred2))
print("classifaction report for log:", classification_report(y_test, pred2))

classifaction report for svc:               precision    recall  f1-score   support

           0       0.54      0.83      0.66        24
           1       0.77      0.82      0.79        28
           2       0.78      0.58      0.67        12
           3       0.78      0.37      0.50        19
           4       0.79      0.68      0.73        28
           5       1.00      1.00      1.00         5
           6       1.00      0.29      0.44         7
           7       0.74      0.87      0.80        23
           8       0.59      0.57      0.58        23
           9       1.00      1.00      1.00         8
          10       0.00      0.00      0.00         4
          11       1.00      1.00      1.00         6
          12       0.62      0.75      0.68        24
          13       0.88      0.92      0.90        24
          14       1.00      1.00      1.00         5
          15       0.86      0.86      0.86        22
          16       0.60      0.13      0.21        

In [50]:
dt = tfidf.transform(["yadav narayan hari  student at university of mumbai  github    hackerrank    linkedin     yashwant nagar  teen dongri near pathak hotel goregaon west mumbai        maharashtra             narayanh     gmail com  experience  skills  ineuron intelligence pvt ltd  bangalore   internship  august        january       during internship with ineuron  they have allotted individual projects to  every one  i have done two projects   news article sorting  and   jobseeker web application       python  django  framework    mysql  database     machine learning   nlp  statistics  deep  learning       gcp  achievements  google facilitator program public pro le    public pro les    aab     a         f    e  fb       platforms    windows       anaconda     jupyter notebook    vs code ide   languages  hindi  english   education  patkar varde college  mumbai  b sc  it   july        july       in b sc it  i have secured a score of          i c  durgaganj bhadohi  uttar pradesh   hsc  july        july       in hsc  i have secured a score of          projects  jobseeker web application   python  django  jobseeker web application provides an easy process to apply for any  jobs or publish any jobs within a single click   github  repo     application link     news article sorting   python  django ml nlp  sort any news article within     or less than   second  this project can sort   ve di erent classes of news  business sports politics technical and  entertainment   github repo     application link "])
res = model_log.predict_proba(dt)

In [51]:
res

array([[0.01434169, 0.038766  , 0.03073561, 0.01840774, 0.06672627,
        0.01241523, 0.01542662, 0.02470728, 0.01989025, 0.01595759,
        0.01001298, 0.01583084, 0.01679387, 0.01863172, 0.00944653,
        0.01945518, 0.03384422, 0.04202594, 0.01242705, 0.02569609,
        0.01423468, 0.03064031, 0.01064725, 0.01175058, 0.02297706,
        0.01095423, 0.01897904, 0.01765499, 0.00908332, 0.0153409 ,
        0.01907901, 0.03083829, 0.02141158, 0.01546999, 0.0186559 ,
        0.00856356, 0.00942805, 0.00528201, 0.04926882, 0.09352387,
        0.01770398, 0.01374539, 0.02415799, 0.01801154, 0.03105896]])

In [53]:
from pprint import pprint
import numpy as np

C = [float(x) for x in np.linspace(start = 0.1, stop = 1, num = 10)]

# multi_class
multi_class = ['multinomial']

# solver
solver = ['newton-cg', 'sag', 'saga', 'lbfgs']
 
# class_weight
class_weight = ['balanced', None]

# penalty
penalty = ['l2']

# Create the random grid
random_grid = {'C': C,
               'multi_class': multi_class,
               'solver': solver,
               'class_weight': class_weight,
               'penalty': penalty}

pprint(random_grid)

{'C': [0.1,
       0.2,
       0.30000000000000004,
       0.4,
       0.5,
       0.6,
       0.7000000000000001,
       0.8,
       0.9,
       1.0],
 'class_weight': ['balanced', None],
 'multi_class': ['multinomial'],
 'penalty': ['l2'],
 'solver': ['newton-cg', 'sag', 'saga', 'lbfgs']}


In [56]:
from sklearn.model_selection import RandomizedSearchCV
model = LogisticRegression()
random_search = RandomizedSearchCV(estimator=model,
                                   param_distributions=random_grid,
                                   n_iter=10,
                                   scoring='accuracy',
                                   cv=3, 
                                   verbose=1, 
                                   random_state=8)

# Fit the random search model
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3, estimator=LogisticRegression(),
                   param_distributions={'C': [0.1, 0.2, 0.30000000000000004,
                                              0.4, 0.5, 0.6, 0.7000000000000001,
                                              0.8, 0.9, 1.0],
                                        'class_weight': ['balanced', None],
                                        'multi_class': ['multinomial'],
                                        'penalty': ['l2'],
                                        'solver': ['newton-cg', 'sag', 'saga',
                                                   'lbfgs']},
                   random_state=8, scoring='accuracy', verbose=1)

In [57]:
random_search.best_params_


{'solver': 'sag',
 'penalty': 'l2',
 'multi_class': 'multinomial',
 'class_weight': 'balanced',
 'C': 1.0}

In [60]:
logm = LogisticRegression(solver='sag',
                          penalty='l2',
                          multi_class='multinomial',
                          class_weight='balanced',
                          C=1.0)

mod_log = OneVsRestClassifier(logm)

In [62]:
mod_log.fit(X_train, y_train)

OneVsRestClassifier(estimator=LogisticRegression(class_weight='balanced',
                                                 multi_class='multinomial',
                                                 solver='sag'))

In [61]:

pred = mod_log.predict(X_train)
pred2 = mod_log.predict(X_test)

print("training accuracy of onevsall model log:",accuracy_score(y_train,pred))
print("testing accuracy of onevsall model log:",accuracy_score(y_test,pred2))

NotFittedError: This OneVsRestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.